<a id='agents'></a>

<a href='#top'>Back to top</a>

## AI agents
-----

Agents extend the functionality of what LLMs can do. A typical LLM is capable of generating text based on its context and training data. In addition, an agent can choose to take action using tools that are made available to it.

The new [OpenAI Assistants API](https://platform.openai.com/docs/assistants/overview) provides the ability to write an agent that has access to OpenAI-hosted tools, like Code Interpreter and Knowledge Retrieval, or to custom tools that you define as functions.

Traditional LLMs are pretty bad at math because of how they work, but an agent can be instructed to write code to solve math problems and thus produce much better results than a typical LLM. Let's go through a simple example that demonstrates how this works.

First, let's show how even the latest GPT-4 Turbo fails to solve basic math problems correctly:

In [ ]:
from openai import OpenAI # pip install openai

openai_client = OpenAI()

def query_gpt4(prompt):  
  response = openai_client.chat.completions.create(
    model="gpt-4-1106-preview", # This is the id of the GPT-4 Turbo model
    messages=[
      {"role": "user", "content": prompt}
    ]
  )
  return response.choices[0].message.content

In [ ]:
# Correct answer is 159.4915146
prompt = "5.423 * 29.4102 = ?"
print(query_gpt4(prompt))

To solve the multiplication of 5.423 and 29.4102, multiply these two numbers together:

5.423 * 29.4102 = 159.5142466


It's close but not quite right, and errors tend to compound, so as you get further into a math-focused chat session, the errors are going to get progressively worse.

To solve this, we can use the ChatGPT Assistants API to instruct GPT to use Code Interpreter to write and run Python code in a sandboxed execution environment to get more accurate answers to these sorts of questions.

In [ ]:
assistant = openai_client.beta.assistants.create(
  instructions="You are a personal math tutor. When asked a math question, write and run code to answer the question.",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}]
)

Next, create a thread with a message containing the math problem you want ChatGPT to solve

In [ ]:
thread = openai_client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "5.423 * 29.4102 = ?"
    }
  ]
)

Finally, run the thread to get the agent's response to your prompt.

In [ ]:
run = openai_client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [ ]:
messages = openai_client.beta.threads.messages.list(thread_id=thread.id)

In [ ]:
messages.data[0].content[0].text.value

'The product of 5.423 and 29.4102 is approximately 159.4915146.'

And this is exactly the correct answer! To recap:

* Original prompt: 5.423 * 29.4102 = ?
* Correct answer: 159.4915146
* Answer w/o agent: 159.5142466 (wrong)
* Answer w/ agent: 159.4915146 (right!)